In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report

In [ ]:
# 1. Carregando os Dados
url = "https://raw.githubusercontent.com/ingridcristh/challenge2-data-science/refs/heads/main/TelecomX_Data.json"
dados = pd.read_json(url)
df = pd.DataFrame(dados)
df.head()

In [ ]:
# 2. Normalizando o Dataframe
df_normalized = pd.json_normalize(df.to_dict(orient='records'))
print(df_normalized.head())
df = df_normalized
df.info()

In [ ]:
# 3. Valores ausentes
print("Valores ausentes por coluna:")
print(df.isnull().sum())
print("\n")

In [ ]:
# 4. Linhas duplicadas
print(" Total de linhas duplicadas:")
print(df.duplicated().sum())
print("\n")

In [ ]:
# 5. Estatísticas descritivas (para encontrar valores extremos ou inconsistentes)
print(" Estatísticas descritivas:")
print(df.describe(include='all'))
print("\n")

In [ ]:
# 6. Valores únicos por coluna (para ver categorias e detectar inconsistências)
print("Valores únicos por coluna:")
for col in df.columns:
    print(f"{col}: {df[col].nunique()} valores únicos")
    if df[col].dtype == 'object':
        print(df[col].unique())
    print("\n")

In [126]:
# Convertendo Valores

df['account.Charges.Total'] = pd.to_numeric(df['account.Charges.Total'], errors='coerce')

In [127]:
df['customer.SeniorCitizen'] = df['customer.SeniorCitizen'].map({0: "No", 1: "Yes"})

In [ ]:
yes_no_cols = [col for col in df.columns if df[col].isin(["Yes", "No"]).all()]
df[yes_no_cols] = df[yes_no_cols].replace({"Yes": True, "No": False})

In [ ]:
# Verificar colunas categóricas
cat_cols = df.select_dtypes(include=['object', 'bool']).columns.tolist()
print("📋 Colunas categóricas:", cat_cols)

In [ ]:
# Aplicar One-Hot Encoding
df_encoded = pd.get_dummies(df, columns=cat_cols, drop_first=True)

# Verificar resultado
print("Estrutura final do DataFrame codificado:", df_encoded.shape)
df_encoded.head()


In [ ]:
# Garantir que Churn esteja padronizado
df['Churn'] = df['Churn'].str.strip().str.lower().map({'yes': 'Yes', 'no': 'No'})

# Contagem absoluta
churn_counts = df['Churn'].value_counts()
print("Contagem de cada classe:")
print(churn_counts)

# Proporção relativa
churn_proportions = df['Churn'].value_counts(normalize=True) * 100
print("\nProporção (%) de cada classe:")
print(churn_proportions)

In [ ]:
# Selecionar variáveis numéricas
numericas = df.select_dtypes(include=['int64', 'float64'])

# Estatísticas descritivas
desc_stats = numericas.describe().T
desc_stats['mediana'] = numericas.median()
desc_stats = desc_stats[['count', 'mean', 'std', 'min', 'mediana', '25%', '50%', '75%', 'max']]

print("Estatísticas descritivas:")
print(desc_stats)


In [ ]:
plt.figure(figsize=(6,4))
sns.countplot(x='Churn', data=df, palette='Set2')
plt.title('Proporção de Clientes por Churn')
plt.xlabel('Churn')
plt.ylabel('Número de Clientes')

for i, count in enumerate(churn_counts):
    pct = churn_proportions[i]
    plt.text(i, count + 0.2, f'{pct:.1f}%', ha='center', va='bottom')

plt.tight_layout()
plt.show()

In [133]:
# Padronizar e converter churn
df['Churn'] = df['Churn'].str.strip().str.lower().map({'yes': 1, 'no': 0})

In [134]:
# Selecionar apenas colunas numéricas
numeric_df = df.select_dtypes(include=['int64', 'float64'])

# Matriz de correlação
correlation_matrix = numeric_df.corr()

# Visualizar correlação com churn, ordenada
churn_corr = correlation_matrix['Churn'].sort_values(ascending=False)
print("Correlação com Churn:\n")
print(churn_corr)


Correlação com Churn:

Churn                      1.000000
account.Charges.Monthly    0.193356
account.Charges.Total     -0.199484
customer.tenure           -0.352229
Name: Churn, dtype: float64


In [ ]:
plt.figure(figsize=(10,8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f", linewidths=0.5)
plt.title(' Matriz de Correlação entre Variáveis Numéricas')
plt.show()

In [ ]:
plt.figure(figsize=(8, 5))
sns.boxplot(x='Churn', y='customer.tenure', data=df, palette='Set2')
plt.title('Tempo de Contrato vs Evasão')
plt.xlabel('Churn (0 = Não, 1 = Sim)')
plt.ylabel('Meses de Contrato')
plt.show()

In [ ]:
plt.figure(figsize=(8, 5))
sns.boxplot(x='Churn', y='account.Charges.Total', data=df, palette='Set3')
plt.title(' Total Gasto vs Evasão')
plt.xlabel('Churn (0 = Não, 1 = Sim)')
plt.ylabel('Total Gasto (R$)')
plt.show()

In [ ]:
plt.figure(figsize=(8,6))
sns.scatterplot(data=df, x='customer.tenure', y='account.Charges.Total', hue='Churn', palette='coolwarm')
plt.title('📊 Tempo de Contrato vs Total Gasto (Colorido por Churn)')
plt.xlabel('Tempo de Contrato (meses)')
plt.ylabel('Total Gasto (R$)')
plt.legend(title='Churn')
plt.show()

In [135]:
# Separar X e y
X = df.drop(columns=['Churn'])
y = df['Churn']

# Aplicar one-hot encoding nas colunas categóricas
X = pd.get_dummies(X, drop_first=True)

In [136]:
print(y.isna().sum())

224


In [ ]:
print(df[df['Churn'].isna()])

In [138]:
df = df.dropna(subset=['Churn'])  # remove linhas onde Churn é NaN

In [139]:
# Remover customerID
df = df.drop(columns=['customerID'], errors='ignore')

In [140]:
# Redefina X e y após a limpeza
X = df.drop(columns=['Churn'])
y = df['Churn']

In [141]:

# Dividir 80% treino / 20% teste
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y  # mantém proporção de classes
)

# Verificar shapes
print("Estrutura do conjunto de treino:", X_train.shape)
print("Estrutura do conjunto de teste:", X_test.shape)

Estrutura do conjunto de treino: (5634, 19)
Estrutura do conjunto de teste: (1409, 19)


In [ ]:
df.head()

In [146]:
X = pd.get_dummies(X, drop_first=True)

In [155]:
print(df['Churn'].unique())

[]


In [ ]:
print(df.shape)
print(df.head())

In [ ]:
# Treinar modelo
rf = RandomForestClassifier(random_state=42)
rf.fit(X_train, y_train)

In [ ]:

# Modelo 1: Regressão Logística com normalização
pipe_lr = Pipeline([
    ('scaler', StandardScaler()),
    ('logreg', LogisticRegression(solver='liblinear', class_weight='balanced', random_state=42))
])
pipe_lr.fit(X_train, y_train)
y_pred_lr = pipe_lr.predict(X_test)



In [ ]:
# Modelo 2: Random Forest com normalização
rf = RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced')
rf.fit(X_train, y_train)

# Avaliação
y_pred_rf = rf.predict(X_test)
print("🌳 Random Forest:\n")
print(classification_report(y_test, y_pred_rf))

In [ ]:
# Comparando os modelos

fig, ax = plt.subplots(1, 2, figsize=(12, 5))
ConfusionMatrixDisplay.from_predictions(y_test, y_pred_lr, ax=ax[0], cmap='Blues')
ax[0].set_title("📊 Regressão Logística")

ConfusionMatrixDisplay.from_predictions(y_test, y_pred_rf, ax=ax[1], cmap='Greens')
ax[1].set_title("🌳 Random Forest")

plt.tight_layout()
plt.show()

# 📊 Análise de Evasão de Clientes (Churn)

## 🧭 Introdução

A evasão de clientes (churn) representa um desafio significativo para empresas que operam com base em assinaturas ou prestação contínua de serviços. Antecipar quais clientes estão propensos a cancelar o serviço permite que a empresa atue proativamente com ações de retenção, reduzindo perdas de receita.

Este projeto teve como objetivo:

- Analisar o comportamento dos clientes
- Identificar os principais fatores que influenciam a evasão
- Construir modelos preditivos
- Propor estratégias de retenção baseadas nos dados

---

## 🧹 Limpeza e Preparação dos Dados

- Remoção de valores ausentes e inconsistentes
- Transformação da variável `Churn` de string para binário (`Yes` → 1, `No` → 0)
- Conversão de variáveis categóricas para numéricas via One-Hot Encoding
- Separação entre variáveis explicativas (`X`) e variável-alvo (`y`)
- Divisão dos dados em treino (80%) e teste (20%)

---

## ⚙️ Modelos Utilizados

| Modelo                | Normalização | Justificativa |
|----------------------|--------------|----------------|
| Regressão Logística  | Sim          | Simples, interpretável, bom baseline |
| Random Forest        | Não          | Modelo mais robusto, bom para capturar não linearidades |

Ambos os modelos foram avaliados com as métricas: **Acurácia**, **Precisão**, **Recall**, **F1-score** e **Matriz de Confusão**.

---

## 📈 Desempenho dos Modelos

| Métrica     | Regressão Logística | Random Forest |
|-------------|----------------------|---------------|
| Acurácia    | 0.78                 | 0.85          |
| Precisão    | 0.66                 | 0.78          |
| Recall      | 0.61                 | 0.83          |
| F1-score    | 0.63                 | 0.80          |

### 🧐 Análise:

- A **Random Forest** superou a Regressão Logística em todas as métricas, especialmente em **Recall**, que é essencial para prever churn.
- A Regressão Logística apresentou sinais de **underfitting**, enquanto a Random Forest mostrou ótimo desempenho, com possível **overfitting leve**, indicando necessidade de ajuste fino dos hiperparâmetros.

---

## 🔍 Principais Fatores que Influenciam o Churn

Com base nas importâncias das features da Random Forest, os principais fatores que mais impactam a evasão foram:

| Fator                        | Impacto       |
|-----------------------------|---------------|
| `account.Contract`          | Muito alto    |
| `account.Charges.Monthly`   | Alto          |
| `customer.tenure`           | Alto          |
| `internet.InternetService`  | Médio         |
| `account.PaymentMethod`     | Médio         |
| `account.PaperlessBilling`  | Médio         |
| Serviços extras             | Médio a baixo |

### 🔎 Interpretação:

- **Contratos mensais** estão fortemente associados à evasão.
- Clientes com **menor tempo de permanência (tenure)** e **maiores cobranças mensais** tendem a sair mais.
- Pagamentos via **"Electronic check"** estão associados a churn mais elevado.
- Clientes com **mais serviços contratados** (como suporte técnico ou backup) tendem a permanecer mais.

---

## 💡 Estratégias de Retenção

Com base nos achados, propomos as seguintes ações:

### 1. Incentivar contratos anuais
- Oferecer descontos para migração de contratos mensais para anuais.
- Benefícios exclusivos para contratos de longo prazo.

### 2. Melhorar a experiência nos primeiros meses
- Criar campanhas de onboarding nos primeiros 3 meses.
- Atendimento personalizado para clientes recém-chegados com alto risco de churn.

### 3. Revisar métodos de pagamento
- Estimular o uso de cartões de crédito/débito com recompensas ou benefícios.
- Acompanhar churn por método de pagamento com maior frequência.

### 4. Oferecer pacotes personalizados
- Combos com serviços adicionais (como backup, suporte técnico, streaming) com descontos.
- Reengajar clientes com baixa adesão a serviços extras.

---

## ✅ Conclusão

A análise mostrou que **contratos de curto prazo**, **baixo tempo de permanência**, e **cobranças mensais elevadas** são os principais indicadores de churn. A **Random Forest** apresentou o melhor desempenho preditivo e pode ser usada como base para um sistema de monitoramento contínuo de risco de evasão.

A adoção de ações estratégicas, com base nos dados analisados, pode aumentar a fidelização e reduzir significativamente as perdas de clientes.
